### 基于图的推荐算法 PersonalRank算法
- 将用户行为转化为图模型
- 哪些不与用户u相连的item节点，对用户u的影响力大？

####  从用户u对应的节点开始游走
## $ PR_\left(v_i\right)  =   \frac{ 1-d }{ N}+  d  \times \sum_{v_j\in V_i}    \frac{PR\left( v_i\right)}{  |(out_j)|}$

## $ PR_\left(v_i\right)  =   \ \left(1-d\right)\times r_i+  d  \times \sum_{v_j\in V_i}    \frac{PR\left( v_i\right)}{  |(out_j)|}$

## $  r_i=\left\{
\begin{aligned}
1 & \   i== u \\
0 & \   i!= u \\ 
\end{aligned}
\right.$
+ PageRank随机模型改成以(1-d)的概率固定从u重新开始
+ d的概率继续游走
+ 当收敛的时候，计算item节点影响力排名，即为用户u感兴趣的item

In [1]:
import numpy as np 

####  1.获得用户商品的打分记录矩阵 ，matrix(i，j )表示user_i  对item_j的行为记录， 1 表示打过分， 0表示没有打过分数

In [2]:
def get_user_item_bool_matrix( path) :
    f = open( path , "r") 
    dataMatrix =[] 
    for line in  f.readlines():
        templist = [] 
        for  e   in line.strip().split("\t") :
            if e!= "-" :
                templist.append( float(e)) 
            else:
                templist.append(0) 
        dataMatrix.append(templist) 

    f.close()
    return  np.array(dataMatrix ,dtype = "float")

In [14]:
def get_user_item_bool_matrixInt( path) :
    f = open( path , "r") 
    dataMatrixInt =[] 
    for line in  f.readlines():
        templist = [] 
        for  e   in line.strip().split("\t") :
            if e!= "-" :
                templist.append( 1) 
            else:
                templist.append(0) 
        dataMatrixInt.append(templist) 

    f.close()
    return  np.array(dataMatrixInt)

In [4]:
dataMatrix= get_user_item_bool_matrix( path = "./data.txt")

In [5]:
dataMatrixInt = get_user_item_bool_matrixInt( path = "./data.txt")

#### 生成user_item二部图

In [7]:
from collections import defaultdict 

def generate_dict_of_useritm(  dataMatrixInt  , dataMatrix ) :
    m , n  =  dataMatrix.shape 
    datadict ={} 
    for  i in range(m) :
        tempdict = {}
        for  j in range(n):
            if(dataMatrixInt[i][j] !=0 ):
                tempdict ["item_" + str(j)  ]  =  dataMatrixInt[ i][j ]
        datadict["user_"+str(i)]  =  tempdict
    
    for  j in range(n) :
        tempdict = {}
        for  i in range(m):
            if( dataMatrixInt[i][j ] != 0) :
                tempdict ["user_" + str(i)  ]  =  dataMatrixInt[ i][j ]
        datadict["item_"+str(j)]  =  tempdict
    
    return  datadict 

In [8]:
g = generate_dict_of_useritm( dataMatrixInt  , dataMatrix )

In [9]:
g

{'user_0': {'item_0': 1.0, 'item_1': 1.0, 'item_3': 1.0},
 'user_1': {'item_0': 1.0, 'item_2': 1.0, 'item_3': 1.0},
 'user_2': {'item_0': 1.0, 'item_2': 1.0, 'item_4': 1.0},
 'user_3': {'item_0': 1.0, 'item_1': 1.0, 'item_3': 1.0},
 'user_4': {'item_1': 1.0, 'item_2': 1.0, 'item_4': 1.0},
 'item_0': {'user_0': 1.0, 'user_1': 1.0, 'user_2': 1.0, 'user_3': 1.0},
 'item_1': {'user_0': 1.0, 'user_3': 1.0, 'user_4': 1.0},
 'item_2': {'user_1': 1.0, 'user_2': 1.0, 'user_4': 1.0},
 'item_3': {'user_0': 1.0, 'user_1': 1.0, 'user_3': 1.0},
 'item_4': {'user_2': 1.0, 'user_4': 1.0}}

In [10]:
def perosonRank( data_dict  , user  , alpha = 0.85  ,maxiter  =100 ,threshold = 1e-5  ) : 
    rank = {}  
    for x in data_dict.keys():
        rank[x] = 0
    rank[user] = 1 # 从user开始游走

    # 2、迭代
    step = 0
    while step < maxiter :
        tmp = {}
        for x in data_dict.keys():
            tmp[x] = 0

        for i, ri in data_dict.items():
            for j in ri.keys():
                if j not in tmp:
                    tmp[j] = 0
                tmp[j] += alpha * rank[i] / (1.0 * len(ri))  
                if j == user:
                    tmp[j] += (1 - alpha)
        # 判断是否收敛
        check = []
        for k in tmp.keys():
            check.append(tmp[k] - rank[k])	
        if sum(check) <=threshold:
            break
        rank = tmp
        if step % 20 == 0:
            print("iter: ", step)
        step = step + 1
    return rank
        


In [11]:
rank  = perosonRank( g  , "user_0", alpha = 0.85  ,maxiter  =100   )

iter:  0
iter:  20
iter:  40
iter:  60


In [12]:
def recommend(data_dict, rank, user):
    items_dict = {}
    # 1、用户user已打过分的项
    items = []
    for k in data_dict[user].keys():
        items.append(k)

    # 2、从rank取出商品的打分
    for k in rank.keys():
        if k.startswith("item_"): # 商品
            if k not in items: # 排除已经互动过的商品
                items_dict[k] = rank[k]

    # 3、按打分的降序排序
    result = sorted(items_dict.items(), key=lambda d: d[1], reverse=True)
    return result

In [13]:
recommend(g, rank, 'user_0')

[('item_2', 0.1712029993978684), ('item_4', 0.10454553424474539)]